In [17]:
import os
import tarfile
import gzip
import shutil
import scanpy as sc # If error, pip install scanpy
import pandas as pd
import re # Allows splitting to remove GSM labels later
from pathlib import Path

In [14]:

base_dir = Path.cwd()
tar_path = base_dir / "GSE159977_RAW.tar"
extract_dir = base_dir / "GSE159977_RAW"
out_dir = base_dir / "GSE159977_h5ad"
out_dir.mkdir(parents=True, exist_ok=True)


In [24]:
# -------------------------------
# Step 0: Define paths
# -------------------------------

base_dir = Path.cwd()  # /home/tsaliu (from your output)

tar_file = "GSE159977_RAW.tar"          # downloaded tar file
extract_folder = "GSE159977_RAW"        # folder to extract files into
output_folder = "GSE159977_h5ad"        # folder to save h5ad files
os.makedirs(output_folder, exist_ok=True)

In [15]:
# -------------------------------
# Step 1: Extract the tar file
# -------------------------------
print("Extracting tar file...")
with tarfile.open(tar_file, "r") as tar:
    tar.extractall(path=extract_folder)
print("Extraction done.")


Extracting tar file...
Extraction done.


In [25]:
# -------------------------------
# Step 2: Unzip all .gz files recursively
# -------------------------------
print("Unzipping .gz files...")
for root, dirs, files in os.walk(extract_folder):
    for f in files:
        if f.endswith(".gz"):
            gz_path = os.path.join(root, f)
            out_path = os.path.join(root, f[:-3])  # remove .gz
            with gzip.open(gz_path, 'rb') as f_in:
                with open(out_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
print("All files unzipped.")

Unzipping .gz files...
All files unzipped.


In [18]:
# -------------------------------
# Step 3: Map GSM to PT labels
# -------------------------------
gsm_to_pt = {
    "GSM4851987": "PT-5",
    "GSM4851988": "PT-8",
    "GSM4851989": "PT-9",
    "GSM4851990": "PT-11",
    "GSM4851991": "PT-12",
    "GSM4851992": "PT-16",
    "GSM4851993": "PT-17",
    "GSM4851994": "PT-19",
    "GSM4851995": "PT-20",
    "GSM4851996": "PT-21"
}

In [19]:
# -------------------------------
# Step 3.5: File Grouping Fix
# -------------------------------

# group files by the GSM...PT-... prefix before "-barcodes/-features/-matrix"
pat = re.compile(r"^(GSM\d+_PT-\d+)-(barcodes|features|matrix)\.(tsv|mtx)(\.gz)?$")

groups = {}
for fn in os.listdir(extract_folder):
    m = pat.match(fn)
    if not m:
        continue
    sample_key = m.group(1)  # e.g. GSM4851987_PT-5
    groups.setdefault(sample_key, []).append(fn)

print("Detected samples:", len(groups))
print("Example:", list(groups.keys())[:3])

# make folders and place files with standard 10x names inside
for sample_key, fns in groups.items():
    sample_dir = os.path.join(extract_folder, sample_key)
    os.makedirs(sample_dir, exist_ok=True)

    for fn in fns:
        src = os.path.join(extract_folder, fn)

        # rename to 10x expected names
        if "-barcodes.tsv" in fn:
            dst_name = "barcodes.tsv" + (".gz" if fn.endswith(".gz") else "")
        elif "-features.tsv" in fn:
            dst_name = "features.tsv" + (".gz" if fn.endswith(".gz") else "")
        elif "-matrix.mtx" in fn:
            dst_name = "matrix.mtx" + (".gz" if fn.endswith(".gz") else "")
        else:
            continue

        dst = os.path.join(sample_dir, dst_name)
        shutil.move(src, dst) 

Detected samples: 10
Example: ['GSM4851987_PT-5', 'GSM4851988_PT-8', 'GSM4851989_PT-9']


In [20]:
# -------------------------------
# Step 4: Identify sample folders
# -------------------------------
sample_folders = [os.path.join(extract_folder, d) 
                  for d in os.listdir(extract_folder) 
                  if os.path.isdir(os.path.join(extract_folder, d))]

print(f"Found {len(sample_folders)} sample folders.")

Found 10 sample folders.


In [21]:
# -------------------------------
# Sanity Check: GSM4851987_PT-5 should contain barcodes.tsv(.gz), features.tsv(.gz), matrix.mtx(.gz)
# -------------------------------

print(os.listdir(os.path.join(extract_folder, "GSM4851987_PT-5")))


['barcodes.tsv.gz', 'features.tsv.gz', 'matrix.mtx.gz', 'barcodes.tsv', 'features.tsv', 'matrix.mtx']


In [22]:
# -------------------------------
# Step 5: Load each sample, save per sample
# -------------------------------
adata_list = []
for folder in sample_folders:
    sample_id = os.path.basename(folder)

    gsm = sample_id.split("_")[0]  # Removes GSM labels (e.g. GSM4851987)
    pt_label = gsm_to_pt.get(gsm, sample_id)

    print(f"Loading sample {sample_id} -> {pt_label}...")

    ad = sc.read_10x_mtx(folder, var_names='gene_symbols', cache=True)

    # Add metadata
    ad.obs['sample'] = pt_label
    ad.obs['dataset'] = 'GSE159977_RAW'

    # Save per-sample h5ad
    per_sample_file = os.path.join(output_folder, f"GSE159977_{pt_label}.h5ad")
    ad.write(per_sample_file)
    print(f"Saved {per_sample_file}")

    adata_list.append(ad)

Loading sample GSM4851992_PT-16 -> PT-16...
Saved GSE159977_h5ad/GSE159977_PT-16.h5ad
Loading sample GSM4851993_PT-17 -> PT-17...
Saved GSE159977_h5ad/GSE159977_PT-17.h5ad
Loading sample GSM4851994_PT-19 -> PT-19...
Saved GSE159977_h5ad/GSE159977_PT-19.h5ad
Loading sample GSM4851995_PT-20 -> PT-20...
Saved GSE159977_h5ad/GSE159977_PT-20.h5ad
Loading sample GSM4851996_PT-21 -> PT-21...
Saved GSE159977_h5ad/GSE159977_PT-21.h5ad
Loading sample GSM4851987_PT-5 -> PT-5...
Saved GSE159977_h5ad/GSE159977_PT-5.h5ad
Loading sample GSM4851988_PT-8 -> PT-8...
Saved GSE159977_h5ad/GSE159977_PT-8.h5ad
Loading sample GSM4851989_PT-9 -> PT-9...
Saved GSE159977_h5ad/GSE159977_PT-9.h5ad
Loading sample GSM4851990_PT-11 -> PT-11...
Saved GSE159977_h5ad/GSE159977_PT-11.h5ad
Loading sample GSM4851991_PT-12 -> PT-12...
Saved GSE159977_h5ad/GSE159977_PT-12.h5ad


In [23]:
print(os.getcwd())

/home/tsaliu
